In [ ]:

import re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


USE_TEX = True
FIGSIZE = (12.5, 7.5)
DPI = 300

MAIN_AX_RECT = [0.08, 0.10, 0.84, 0.85]

CSV_PFF = "noise_isi_cluster_0.06.csv"

PSD_PATTERN = "node_24_n_*_c_00600_.csv"

TAU = 431.0
FMAX_PLOT = 0.02          # frequency band (1/time)
XMAX_MULT = FMAX_PLOT * TAU

PSD_YMIN = 1e-10
PSD_YMAX = 1e6

PALETTE = {
    0: "#e41a1c",  # red
    1: "#377eb8",  # blue
    2: "#4daf4a",  # green
    3: "#984ea3",  # purple
    4: "#ff7f00",  # orange
}

PSD_COLOR = {
    "(a)": "#984ea3",  # purple
    "(b)": "#4daf4a",  # green
    "(c)": "#4daf4a",  # green
    "(d)": "#e41a1c",  # red
}

INSET_W_FIG = 0.26   # width in FIGURE fraction
INSET_H_FIG = 0.16   # height in FIGURE fraction

INSET_ANCHORS = {
    "(a)": (6e-4, 0.2),
    "(b)": (6e-4, 0.42),
    "(c)": (2.3e-1, 0.07),
    "(d)": (1.2e-1, 0.42),
}

INSET_MM_OFFSETS = {
    "(a)": (0.0, 0.0),
    "(b)": (0.0, 0.0),
    "(c)": (0.0, 0.0),
    "(d)": (0.0, -2.0),
}

DEFAULT_LABEL_OFFSETS = {
    "(a)": (1, 26),
    "(b)": (-40, 26),
    "(c)": (0, -66),
    "(d)": (-55, 1),
}

DESIRED_N = {
    "(a)": 0.0002,
    "(b)": 0.0012,
    "(c)": 0.0200,
    "(d)": 0.4000,
}


def extract_n_tag(path: Path) -> str:
    m = re.search(r"n_(\d+)_c", path.name)
    return m.group(1) if m else "00000"

def n_from_tag(tag: str) -> float:
    try:
        return round(int(tag) / 1e4, 4)
    except Exception:
        return 0.0

def read_flex_csv(p: Path) -> pd.DataFrame:
    try:
        df = pd.read_csv(p)
        if df.shape[1] == 1 and str(df.columns[0]).startswith("Unnamed"):
            df = pd.read_csv(p, sep=None, engine="python")
        return df
    except Exception:
        return pd.read_csv(p, header=None)

def choose_time_and_value_cols(df: pd.DataFrame):
    num = df.select_dtypes(include=[np.number])
    time_col = None
    for c in df.columns:
        lc = str(c).lower()
        if ("time" in lc) or (lc == "t") or lc.startswith("t_"):
            time_col = c
            break
    if time_col is None and len(num.columns) > 0:
        c0 = num.columns[0]
        v = df[c0].to_numpy()
        f = np.isfinite(v)
        if f.sum() > 3 and np.all(np.diff(v[f]) >= 0):
            time_col = c0
    val_cols = [c for c in num.columns if c != time_col]
    if not val_cols:
        last = df.columns[-1]
        df[last] = pd.to_numeric(df[last], errors="coerce")
        val_cols = [last]
    return time_col, val_cols[0]

def uniform_dt(t: np.ndarray):
    if t.size < 3: return None
    dt = np.diff(t)
    dt = dt[np.isfinite(dt)]
    if dt.size == 0: return None
    if np.nanstd(dt) < (1e-6 + 1e-3 * max(np.nanmean(dt), 1.0)):
        return float(np.nanmean(dt))
    return None

def next_pow2(n: int) -> int:
    return int(2 ** np.ceil(np.log2(max(1, n))))

def periodogram_onesided(y: np.ndarray, dt: float):
    y = np.nan_to_num(y - np.nanmean(y))
    N = y.size
    nfft = next_pow2(N)
    w = np.hanning(N)
    U = (w**2).sum() / N
    Y = np.fft.rfft(y * w, n=nfft)
    freqs = np.fft.rfftfreq(nfft, d=dt)
    psd = (dt / (N * U)) * (np.abs(Y) ** 2)
    if psd.size > 2:
        psd[1:-1] *= 2.0
    return freqs, psd

def spectrum_of_csv(path: Path):
    df = read_flex_csv(path)
    t_col, y_col = choose_time_and_value_cols(df)
    y = df[y_col].astype(float).to_numpy()
    t = df[t_col].astype(float).to_numpy() if t_col else np.arange(len(y))
    dt = uniform_dt(t)
    if dt is None:
        return None
    freqs, psd = periodogram_onesided(y, dt)
    nval = n_from_tag(extract_n_tag(path))
    return {"freqs": freqs, "psd": psd, "n": nval, "path": path}

def pick_spec_by_n(specs, target_n):
    if not specs: return None
    arr = np.array([s["n"] for s in specs])
    idx = int(np.argmin(np.abs(arr - target_n)))
    return specs[idx]


def sci_math_label(val: float) -> str:
    if val <= 0:
        return f"{val:g}"

    exp = int(np.floor(np.log10(val)))
    mant = val / (10.0 ** exp)

    if abs(mant - 1.0) < 1e-8:
        return rf"$10^{{{exp}}}$"

    if abs(mant - round(mant)) < 1e-8:
        return rf"${int(round(mant))}\times 10^{{{exp}}}$"

    return rf"${mant:.1f}\times 10^{{{exp}}}$"


def data_to_fig_xy(ax, x_data, y_data):
    fig = ax.figure
    fig.canvas.draw()
    x_disp, y_disp = ax.transData.transform((x_data, y_data))
    x_fig, y_fig = fig.transFigure.inverted().transform((x_disp, y_disp))
    return float(x_fig), float(y_fig)

def inset_rect_from_anchor(ax, label):
    D_anchor, Y_anchor = INSET_ANCHORS[label]
    x_fig, y_fig = data_to_fig_xy(ax, D_anchor, Y_anchor)

    x0 = x_fig - INSET_W_FIG / 2.0
    y0 = y_fig - INSET_H_FIG / 2.0

    dx_mm, dy_mm = INSET_MM_OFFSETS.get(label, (0.0, 0.0))
    dx_fig = dx_mm / 25.4 / FIGSIZE[0]
    dy_fig = dy_mm / 25.4 / FIGSIZE[1]

    x0 += dx_fig
    y0 += dy_fig

    x0 = np.clip(x0, 0.0, 1.0 - INSET_W_FIG)
    y0 = np.clip(y0, 0.0, 1.0 - INSET_H_FIG)

    return [x0, y0, INSET_W_FIG, INSET_H_FIG]


def draw_psd(ax_psd, spec, color):
    x_mult = spec["freqs"] * TAU
    ax_psd.semilogy(x_mult, spec["psd"], color=color, linewidth=0.9)
    ax_psd.set_xlim(0, XMAX_MULT)
    xticks = np.arange(0, int(np.floor(XMAX_MULT)) + 1, 1)
    ax_psd.set_xticks(xticks)
    ax_psd.set_xticklabels([str(int(x)) for x in xticks], fontsize=18)
    ax_psd.tick_params(axis="both", which="both", direction="in", length=3, pad=2)
    for spine in ax_psd.spines.values():
        spine.set_linewidth(0.8)
    ax_psd.grid(True, linestyle="--", color="gray", alpha=0.3)

'''
def set_psd_ylim(ax_psd, lo, hi):
    ax_psd.set_ylim(lo, hi)
    ax_psd.set_yticks([lo, hi])
    ax_psd.set_yticklabels([f"{lo:.0e}", f"{hi:.0e}"], fontsize=18)
'''
def set_psd_ylim(ax_psd, lo, hi):
    ax_psd.set_ylim(lo, hi)
    ax_psd.set_yticks([lo, hi])
    ax_psd.set_yticklabels(
        [sci_math_label(lo), sci_math_label(hi)],
        fontsize=18
    )

def label_inset(ax_psd, lab):
    ax_psd.text(
        0.02, 0.98, lab,
        transform=ax_psd.transAxes,
        ha="left", va="top",
        fontsize=18, fontweight="bold",
        bbox=dict(facecolor="white", alpha=0.6, edgecolor="none", pad=1.5),
        zorder=10,
    )

def nearest_index_logaware(D, target):
    x = np.asarray(D)
    return int(np.argmin(np.abs(np.log(x) - np.log(target))))


def main():
    df = pd.read_csv(CSV_PFF)
    D = df["noise"].to_numpy()
    psi = df["isi_distance"].to_numpy()
    clusters = df["cluster"].astype(int).to_numpy()
    pt_colors = [PALETTE.get(k, "#808080") for k in clusters]

    plt.rcParams.update({
        "text.usetex": USE_TEX,
        "font.family": "serif",
        "axes.linewidth": 1.2,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "xtick.major.size": 6,
        "ytick.major.size": 6,
        "xtick.minor.size": 3,
        "ytick.minor.size": 3,
        "legend.frameon": True,
        "legend.edgecolor": "black",
    })

    fig = plt.figure(figsize=FIGSIZE, dpi=DPI)
    ax = fig.add_axes(MAIN_AX_RECT)

    ax.plot(D, psi, linestyle="--", color="black", linewidth=1.5)
    ax.scatter(D, psi, s=100, c=pt_colors, edgecolors="black", linewidths=0.5)

    ax.set_xscale("log")
    ax.set_xticks([1e-4, 1e-3, 1e-2, 1e-1])
    ax.set_xticklabels(
        [r"$10^{-4}$", r"$10^{-3}$", r"$10^{-2}$", r"$10^{-1}$"],
        fontsize=22
    )

    ax.set_xlabel(r"$D$", fontsize=24)
    ax.set_xlim(8e-5,10e-1)
    ax.set_ylabel(r"$\omega$", fontsize=32, rotation=0, y=0.61)
    ax.set_ylim(-0.05, 0.55)
    ax.tick_params(axis="y", labelsize=22)
    ax.grid(True, which="both", linestyle="dotted", linewidth=0.8)

    files = sorted(Path(".").glob(PSD_PATTERN),
                   key=lambda p: n_from_tag(extract_n_tag(p)))
    specs_all = [spectrum_of_csv(p) for p in files]
    specs = [s for s in specs_all if s is not None]

    spec_for = {lab: pick_spec_by_n(specs, DESIRED_N[lab]) for lab in DESIRED_N}

    inset_axes = {}
    for lab in ["(a)", "(b)", "(c)", "(d)"]:
        rect = inset_rect_from_anchor(ax, lab)
        iax = fig.add_axes(rect, zorder=4)
        inset_axes[lab] = iax

        spec = spec_for[lab]
        draw_psd(iax, spec, PSD_COLOR[lab])
        set_psd_ylim(iax, PSD_YMIN, PSD_YMAX)
        label_inset(iax, lab)

        iax.set_ylabel("PSD", fontsize=18, rotation=0, labelpad=-20)


    inset_axes["(a)"].set_xlabel(r"$1/T$", fontsize=20, labelpad=2)
    inset_axes["(b)"].set_xlabel(r"$1/T$", fontsize=20, labelpad=2)
    inset_axes["(c)"].set_xlabel(r"$1/T$", fontsize=20, labelpad=2)
    inset_axes["(d)"].set_xlabel(r"$1/T$", fontsize=20, labelpad=2)

    for lab, n_val in DESIRED_N.items():
        idx = nearest_index_logaware(D, n_val)
        xy = (D[idx], psi[idx])
        dx, dy = DEFAULT_LABEL_OFFSETS[lab]
        ax.annotate(
            lab, xy=xy, xycoords="data",
            xytext=(dx, dy), textcoords="offset points",
            ha="left", va="bottom", fontsize=24, color="black", alpha=0.9,
            arrowprops=dict(arrowstyle="->", color="black", alpha=0.7,
                            lw=4.0, mutation_scale=28),
            zorder=6, annotation_clip=False
        )

    out = "pff_with_psd_anchors.png"
    fig.savefig(out, dpi=DPI, bbox_inches="tight")
    print(f"Saved {out}")

if __name__ == "__main__":
    main()

